In [ ]:
import json
import pandas as pd
from rich import print as pp
from icecream import ic
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from rich.progress import track
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

with open('../hf_token.txt', 'r') as file_reader:
    hf_token = file_reader.readlines()[0]

base_df = pd.read_csv('./results/0_base_data_no_result.csv')

# pp(hf_token)

model_names = ['microsoft/Phi-4-mini-instruct',
               'meta-llama/Llama-3.2-3B-Instruct',
               'google/gemma-3-4b-it',
               'Qwen/Qwen3-4B']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# col_names = ['s_a_id', 'q_id', 'label'] + model_names

# df = pd.DataFrame(columns = col_names)

In [4]:
# base_df

In [5]:
# for i in base_df:
#     base_df[i] = base_df[i].astype(str)
#     pp(base_df[i].dtype)

In [6]:
for model_name in model_names:

    # ic(model_name)
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        device_map = device, 
        torch_dtype = "auto", 
        trust_remote_code = True,
        token = hf_token
    )

    # ic(model)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # ic(tokenizer)
    m_pred = []
    # idx = 1

    # for s_a_id in betl2train:
    for s_a_id, _ in track(betl2train.items(), description = "Processing samples"):

        prompt = f'''You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *only* the single word grade ('correct' or 'incorrect') and nothing else.\n\nQuestion: {betl2train[s_a_id]['q']}\n'''

        r_a = ""
        for i in range(len(betl2train[s_a_id]['r_a'])):
            r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''
        
        prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nGrade: '''

        # ic(prompt)

        text_tokenized = tokenizer(prompt, return_tensors = 'pt').to(device)
        tokens_to_gen = 2

        output = model.generate(
            text_tokenized['input_ids'],
            do_sample = True,
            top_p = 0.95,
            temperature = 0.001,
            top_k = 0,
            max_new_tokens = tokens_to_gen)
        
        model_op = tokenizer.decode(output[0])
        model_pred = model_op[len(prompt):].strip().replace('<|endoftext|>', '').lower()
        # ic(model_op)
        # ic(model_pred)
        m_pred.append(model_pred)
        # base_df.loc[str(int(s_a_id) + 1), model_name] = model_pred

        # ic(idx)
        # idx += 1
        # if idx == 20:
        #     break
    
    base_df[model_name] = m_pred




    ic(torch.cuda.memory_allocated())
    del model
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    ic(torch.cuda.memory_allocated())

    break

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Output()

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ic| torch.cuda.memory_allocated(): 7680568320
ic| torch.cuda.memory_allocated(): 8524288


In [7]:
base_df

,s_a_id,q_id,label,microsoft/Phi-4-mini-instruct,google/gemma-3-4b-it,meta-llama/Llama-3.2-3B-Instruct,Qwen/Qwen3-4B
0,1,1,correct,incorrect,NaN,NaN,NaN
1,2,1,correct,correct,NaN,NaN,NaN
2,3,1,incorrect,incorrect,NaN,NaN,NaN
3,4,1,incorrect,correct,NaN,NaN,NaN
4,5,1,incorrect,incorrect,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3936,3937,47,correct,correct,NaN,NaN,NaN
3937,3938,47,incorrect,correct,NaN,NaN,NaN
3938,3939,47,incorrect,correct,NaN,NaN,NaN
3939,3940,47,incorrect,incorrect,NaN,NaN,NaN


['tp', 'fn']
['fp', 'tn']

In [23]:
actual_values = base_df['label']
predicted_values = base_df['microsoft/Phi-4-mini-instruct']
conf_matrix = confusion_matrix(actual_values, predicted_values)
x = [['tp', 'fn'],['fp', 'tn']]
pp(*x, sep = '\n')
pp(conf_matrix)

['tp', 'fn']
['fp', 'tn']

[[1247  418]
 [ 682 1594]]

In [9]:
# conf_matrix

In [24]:
x = [['tp', 'fn'],['fp', 'tn']]
pp(*x, sep = '\n')
base_df.groupby(['label', 'microsoft/Phi-4-mini-instruct']).size().unstack(fill_value=0)

['tp', 'fn']
['fp', 'tn']

microsoft/Phi-4-mini-instruct,correct,incorrect
label,,
correct,1247,418
incorrect,682,1594


In [ ]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from rich.progress import track
import torch

torch.random.manual_seed(666)

with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

model_op = dict()

model_name = "Qwen/Qwen3-4B"
model_op[model_name] = []


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = "auto",
    device_map = "auto"
)

counter = 0

for s_a_id, _ in track(betl2train.items(), description = "Processing samples"):

    prompt = f'''Question: {betl2train[s_a_id]['q']}\n'''

    r_a = ""
    for i in range(len(betl2train[s_a_id]['r_a'])):
        r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''

    prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nWhat is the Student's grade?'''

    messages = [
        {"role": "system", "content": "You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *EXACTLY ONLY ONLY ONE SINGLE WORD GRADE* that is either 'correct' or 'incorrect' and *NOTHING ELSE*."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True,
        enable_thinking = False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=32768
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

    try:
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    model_op[model_name].append(content)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Output()

In [ ]:
print(len(model_op[model_name]))

with open('./results/t0_qwen_no_thought_results.txt', 'w') as f:
    for line in model_op[model_name]:
        f.write(f"{str(line).strip()}\n")

3941


In [ ]:
import json
from transformers import pipeline
import torch
from rich import print as pp

torch.random.manual_seed(666)

with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

model_op = dict()

# model_name = "google/gemma-3-4b-it"
# model_name = "meta-llama/Llama-3.2-3B-Instruct"
model_name = "microsoft/Phi-4-mini-instruct"

model_op[model_name] = []

pipe = pipeline(
    "text-generation",
    model = model_name,
    device = "cuda",
    torch_dtype = torch.bfloat16,
)

m = []


for cnt, s_a_id in enumerate(betl2train):

    prompt = f'''Question: {betl2train[s_a_id]['q']}\n'''

    r_a = ""
    for i in range(len(betl2train[s_a_id]['r_a'])):
        r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''

    prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nWhat is the Student's grade?'''

    messages = [
        {"role": "system", "content": "You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *EXACTLY ONLY ONLY ONE SINGLE WORD GRADE* that is either 'correct' or 'incorrect' and *NOTHING ELSE*."},
        {"role": "user", "content": prompt}
    ]

    m.append(messages)

    if cnt == 10:
        break

output = pipe(m,
              do_sample = True,
              max_new_tokens = 2,
              return_full_text = False,
              temperature =  0.1,)
            #   batch_size = 64)


# pp([op[0]["generated_text"][-1]["content"] for op in output])
pp([op[0]["generated_text"] for op in output])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


[
    'correct',
    'correct',
    'incorrect',
    'correct',
    'incorrect',
    'incorrect',
    'incorrect',
    'correct',
    'correct',
    'incorrect',
    'incorrect'
]

In [3]:
pp([op[0]["generated_text"] for op in output])

[
    'correct\n',
    'correct\n',
    'correct\n',
    'correct\n',
    'correct\n',
    'incorrect\n',
    'incorrect\n',
    'correct\n',
    'correct\n',
    'correct\n',
    'correct\n'
]

In [2]:
pp(output)

[
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'incorrect\n'}],
    [{'generated_text': 'incorrect\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}],
    [{'generated_text': 'correct\n'}]
]

In [ ]:
import json
from transformers import pipeline
import torch

torch.random.manual_seed(666)

with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

model_op = dict()

model_name = "google/gemma-3-4b-it"
model_op[model_name] = []

pipe = pipeline(
    "text-generation",
    model = model_name,
    device = "cuda",
    torch_dtype = torch.bfloat16,
)

m = []
for s_a_id in betl2train:

    prompt = f'''Question: {betl2train[s_a_id]['q']}\n'''

    r_a = ""
    for i in range(len(betl2train[s_a_id]['r_a'])):
        r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''

    prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nWhat is the Student's grade?'''

    messages = [
        {"role": "system", "content": "You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *EXACTLY ONLY ONLY ONE SINGLE WORD GRADE* that is either 'correct' or 'incorrect' and *NOTHING ELSE*."},
        {"role": "user", "content": prompt}
    ]

    m.append(messages)

output = pipe(m,
              do_sample = True,
              max_new_tokens = 2,
              batch_size = 64)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
print(len(output))

model_op[model_name] = [op[0]["generated_text"][-1]["content"] for op in output]

with open('./results/t0_gemma_results.txt', 'w') as f:
    for line in model_op[model_name]:
        f.write(f"{str(line).strip()}\n")

3941


In [ ]:
import torch
from transformers import pipeline
from transformers.utils import logging
logging.set_verbosity_error()

import json

torch.random.manual_seed(666)

with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model = model_id,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
)

m = []

for s_a_id in betl2train:

    prompt = f'''Question: {betl2train[s_a_id]['q']}\n'''

    r_a = ""
    for i in range(len(betl2train[s_a_id]['r_a'])):
        r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''

    prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nWhat is the Student's grade?'''

    messages = [
        {"role": "system", "content": "You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *EXACTLY ONLY ONE SINGLE WORD GRADE* that is either 'correct' or 'incorrect' and *NOTHING ELSE*. *NO EXPLANATIONS ARE NECCESARY*"},
        {"role": "user", "content": prompt}
    ]

    m.append(messages)

outputs = pipe(
    m,
    max_new_tokens = 2,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(len(outputs))

model_op = [op[0]["generated_text"][-1]['content'].lower().strip() for op in outputs]

with open('./results/t0_llama_results.txt', 'w') as f:
    for line in model_op:
        f.write(f"{str(line).strip()}\n")

3941


In [ ]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

torch.random.manual_seed(666)

with open('./data/betl2train_with_s_a_id.json', 'r') as file_reader:
    betl2train = json.load(file_reader)

model_op = dict()

model_path = "microsoft/Phi-4-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.padding_side = "left"

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
)
 
generation_args = {
    "max_new_tokens": 2,
    "return_full_text": False,
    "temperature": 0.1,
    "do_sample": True,
}

m = []
for s_a_id in betl2train:

    prompt = f'''Question: {betl2train[s_a_id]['q']}\n'''

    r_a = ""
    for i in range(len(betl2train[s_a_id]['r_a'])):
        r_a = r_a + f'''\nReference Answer {i + 1}: {betl2train[s_a_id]['r_a'][i]}\n'''

    prompt = prompt + r_a + f'''\nStudent Answer: {betl2train[s_a_id]['s_a']}\n\nWhat is the Student's grade?'''

    messages = [
        {"role": "system", "content": "You are an expert grader. You will be provided with a question, a set of reference answers which should be considered the golden standard, and a student's answer. Your task is to grade the student's answer as 'correct' or  'incorrect' based *only* on its semantic alignment with the reference answer. Output *EXACTLY ONLY ONE SINGLE WORD GRADE* that is either 'correct' or 'incorrect' and *NOTHING ELSE*. *NO EXPLANATIONS ARE NECCESARY*"},
        {"role": "user", "content": prompt}
    ]

    m.append(messages)

output = pipe(m,
              **generation_args,
              batch_size = 64)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
print(len(output))

model_op[model_path] = [op[0]['generated_text'] for op in output]

with open('./results/t0_phi_results.txt', 'w') as f:
    for line in model_op[model_path]:
        f.write(f"{str(line).strip()}\n")

3941
